# The Scalar 1-Loop, 3-Point Vertex Function at Finite Temperature

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from sympy import expand, I, im, symbols, re

import correlator as corr

In [2]:
eps = symbols("eps")

In [4]:
qq_grid = np.arange(-4, 1)
masses = 1, 2, 3
beta_grid = np.array([0.3, 1])
n_max = 4
n_grid = np.arange(-n_max, n_max + 1)

In [5]:
finite_temp_data_Pi =\
    np.array([[[corr.finite_temp_term(*corr.p1_p2(qq),*masses, beta, n) for n in n_grid] for qq in qq_grid] for beta in beta_grid])

In [17]:

finite_temp_data_Gamma = np.zeros(finite_temp_data_Pi.size, dtype="complex128").reshape(finite_temp_data_Pi.shape)
finite_temp_data_re_Gamma = np.zeros(finite_temp_data_Gamma.size, dtype="float64").reshape(finite_temp_data_Gamma.shape)
finite_temp_data_im_Gamma = np.copy(finite_temp_data_re_Gamma)
floors, rows, cols = finite_temp_data_Gamma.shape
for r in range(rows):
    for c in range(cols):
        for f in range(floors):
            finite_temp_data_Gamma[f, r, c] = expand(I * finite_temp_data_Pi[f, r, c].subs("eps", 0))
            finite_temp_data_re_Gamma[f, r, c] = re(finite_temp_data_Gamma[f, r, c])
            finite_temp_data_im_Gamma[f, r, c] = im(finite_temp_data_Gamma[f, r, c])

In [23]:
# cols is an odd, positive integer
partial_sums_Gamma = np.zeros(floors * rows * ((cols + 1) // 2), dtype="complex128").reshape(floors, rows, (cols + 1) // 2)
partial_sums_re_Gamma = np.zeros(partial_sums_Gamma.size).reshape(partial_sums_Gamma.shape)
partial_sums_im_Gamma = np.copy(partial_sums_re_Gamma)
for f in range(floors):
    partial_sums_Gamma[f, :, 0] = finite_temp_data_Gamma[f, :, n_max]
    partial_sums_re_Gamma[f, :, 0] = np.array([re(x) for x in partial_sums_Gamma[f, :, 0]])
    partial_sums_im_Gamma[f, :, 0] = np.array([im(x) for x in partial_sums_Gamma[f, :, 0]])
    for c in range(1, partial_sums_Gamma.shape[1]):
        partial_sums_Gamma[f, :, c] = partial_sums_Gamma[f, :, c - 1]\
            + finite_temp_data_Gamma[f, :, n_max - c] + finite_temp_data_Gamma[f, :, n_max + c]
        partial_sums_re_Gamma[f, :, c] = np.array([re(x) for x in partial_sums_Gamma[f, :, c]])
        partial_sums_im_Gamma[f, :, c] = np.array([im(x) for x in partial_sums_Gamma[f, :, c]])

## Sequence Plots

In [ ]:
selected_qq_grid_indices = [0, 5, 10, 20, 25, 30, -1]

In [ ]:
fig, ax = plt.subplots()
for r in selected_qq_grid_indices:
    ax.plot(n_grid, 1e4 * finite_temp_data_re_Gamma[r, :], label=r"$q^2=$" + str(qq_grid[r]))
ax.set_xlabel("n")
ax.set_ylabel(r"$\mathrm{Re}\Gamma_n \times 10^4$")
ax.legend(loc="upper right")

In [ ]:
fig, ax = plt.subplots()
for r in selected_qq_grid_indices:
    ax.plot(n_grid, 1e4 * finite_temp_data_im_Gamma[r, :], label=r"$q^2=$" + str(qq_grid[r]))
ax.set_xlabel("n")
ax.set_ylabel(r"$\mathrm{Im}\Gamma_n \times 10^4$")
ax.legend(loc="upper right")

## Partial Sums Plots

In [ ]:
fig, ax = plt.subplots()
for r in selected_qq_grid_indices:
    ax.plot(n_grid[n_max:], 1e4 * partial_sums_re_Gamma[r, :],
    label=r"$q^2=$" + str(qq_grid[r]), marker=".")
ax.set_xlabel("n")
ax.set_ylabel(r"$\mathrm{Re}S_n \times 10^4$")
ax.legend(loc="lower right")

In [ ]:
fig, ax = plt.subplots()
for r in selected_qq_grid_indices:
    ax.plot(n_grid[n_max:], 1e4 * partial_sums_im_Gamma[r, :],
    label=r"$q^2=$" + str(qq_grid[r]), marker=".")
ax.set_xlabel("n")
ax.set_ylabel(r"$\mathrm{Im}S_n \times 10^4$")
ax.legend(loc="lower left")

## The Vertex Function

In [ ]:
fig, ax = plt.subplots()
ax.plot(qq_grid, 1e4 * partial_sums_re_Gamma[:, -1], marker=".", color="blue", label=r"$\mathrm{Re}\Gamma$")
ax.plot(qq_grid, 1e4 * partial_sums_im_Gamma[:, -1], marker=".", color="red", label=r"$\mathrm{Im}\Gamma$")
ax.set_xlabel(r"$q^2$")
ax.set_ylabel(r"$\Gamma(q^2)\times 10^4$")
ax.set_title(r"The vertex function at $\beta=${} the symmetric point".format(beta))
ax.legend(loc="upper right")